In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk import PorterStemmer
import joblib
from sklearn.model_selection import KFold ,cross_val_score
import numpy as np

# Dataframe

In [2]:
# import torch
# from transformers import BertTokenizer, BertForSequenceClassification
# # Load the pre-trained BERT model and tokenizer
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# # model = BertForSequenceClassification.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
feedback_data=pd.read_excel('feedback_categorize.xlsx')

In [4]:
feedback_data=feedback_data.drop(['SrNum','Category'],axis=1)
feedback_data=feedback_data.drop_duplicates()
feedback_data=feedback_data.dropna()
feedback_data['Feedback_copy']=feedback_data['Feedback']

In [5]:
feedback_data.shape

(598, 3)

In [6]:
feedback_data.feedback_category.value_counts()

feedback_category
training         68
office           68
management       68
compensation     68
workload         67
opportunities    67
values           65
culture          64
team             63
Name: count, dtype: int64

# Sentiment

In [56]:
from transformers import pipeline
sentiment_pipeline=pipeline('sentiment-analysis')
data=["I enjoy working here; the team is fantastic","The workload is overwhelming, and I need more support","Communication within the team needs improvement","The cafeteria food could be improved",'workload is hectic']
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998836517333984},
 {'label': 'NEGATIVE', 'score': 0.9986737966537476},
 {'label': 'NEGATIVE', 'score': 0.9931689500808716},
 {'label': 'NEGATIVE', 'score': 0.9950055480003357},
 {'label': 'NEGATIVE', 'score': 0.9976702332496643}]

In [61]:
def sentiment_category(text):
    result=sentiment_pipeline(text)[0]
    return result['label']

# Preprocessing

In [9]:
v=TfidfVectorizer()

In [10]:
def preprocess(text):
    text=''.join([char for char in text if char not in string.punctuation])
    text=text.lower()
    tokens=text.split()

    stop_words=set(stopwords.words('english'))
    tokens=[word for word in tokens if word not in stop_words]

    stemm=PorterStemmer()
    # stemm=WordNetLemmatizer()
    stemm_tokens=[stemm.stem(token) for token in tokens]


    cleaned_text=' '.join(stemm_tokens)
    return cleaned_text

In [11]:
feedback_data['Feedback']=feedback_data['Feedback'].apply(preprocess)
# feedback_data.head(20)

In [68]:

feedback_data['sentiment']=feedback_data['Feedback'].apply(sentiment_category)
feedback_data.to_excel("feedback_categorize_output.xlsx", index=False)

In [ ]:
joblib.dump(sentiment_pipeline,'sentiment.pkl')

['sentiment.pkl']

In [69]:
feedback_data.head(10)

,Feedback,feedback_category,Feedback_copy,category_num,sentiment
0,enjoy work team fantast,team,I enjoy working here; the team is fantastic,0,POSITIVE
1,workload overwhelm need support,workload,"The workload is overwhelming, and I need more ...",1,NEGATIVE
2,compani mission align valu,values,The company's mission aligns with my values,4,POSITIVE
3,commun within team need improv,team,Communication within the team needs improvement,0,NEGATIVE
4,appreci flexibl work schedul,workload,I appreciate the flexibility in my work schedule,1,NEGATIVE
5,train program help career growth,training,The training programs are helpful for career g...,6,POSITIVE
6,feel undervalu undercompens,workload,I feel undervalued and undercompensated,1,NEGATIVE
7,offic environ clean comfort,office,The office environment is clean and comfortable,8,POSITIVE
8,concern recent chang manag,management,I have concerns about the recent changes in ma...,5,NEGATIVE
9,love opportun profession develop,opportunities,I love the opportunities for professional deve...,3,POSITIVE


In [13]:
x = v.fit_transform(feedback_data['Feedback'])

In [14]:
joblib.dump(v,'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [15]:
feedback_data['category_num']=feedback_data.feedback_category.map({
    'team':0,
    'workload':1,
    'culture':2,
    'opportunities':3,
    'values':4,
    'management':5,
    'training':6,
    'compensation':7,
    'office':8
})

# feedback_data['Category_sentiment_num']=feedback_data.Category.map({
#     'Negative':0,
#     'Positive':1,
#     'Constructive':2
# })

# Train Test Split

In [16]:
x_train,x_test,y_train,y_test=train_test_split(
    x,
    feedback_data.category_num,
    test_size=0.2,
    random_state=42,
    stratify=feedback_data.category_num
)

In [17]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [34]:
k=60
kf=KFold(n_splits=k,shuffle=True,random_state=42)

# Random Forest Classifier

In [39]:
rf = RandomForestClassifier(n_estimators=200)
rf.fit(x_train,y_train)
results_rf=cross_val_score(rf,x_train,y_train,cv=kf,scoring='accuracy')

average_accuracy_rf=np.mean(results_rf)
print(f"Average Random Forest Accuracy on training: {average_accuracy_rf:.2f}")


y_predict=rf.predict(x_test)
accuracy_score_rf=accuracy_score(y_test,y_predict)
print(f'testing accuracy random forest:  {accuracy_score_rf:.2f}')

Average Random Forest Accuracy on training: 0.78
testing accuracy random forest:  0.79


In [40]:
print("shape of X_train:",x_train.shape)
print("shape of x_test:",x_test.shape)

shape of X_train: (478, 589)
shape of x_test: (120, 589)


In [41]:
y_train.value_counts()

category_num
8    54
1    54
3    54
7    54
6    54
5    54
4    52
0    51
2    51
Name: count, dtype: int64

In [42]:
y_test.value_counts()

category_num
8    14
5    14
7    14
6    14
4    13
2    13
3    13
1    13
0    12
Name: count, dtype: int64

# Dummy data

In [70]:
dummy_data_input=input()
dummy_data=pd.DataFrame({'Feedback':[dummy_data_input]})
dummy_data['Feedback']=dummy_data['Feedback'].apply(preprocess)

dummy_tfidf=v.transform(dummy_data['Feedback'])
# dummy_data['Feedback']



In [71]:
y_predict_dummy=rf.predict(dummy_tfidf)
print(y_predict_dummy)

[1]


In [72]:
feedback_category_dict = {
    0: 'team',
    1: 'workload',
    2: 'culture',
    3: 'opportunities',
    4: 'values',
    5: 'management',
    6: 'training',
    7: 'compensation',
    8: 'office'
}

predicted_category = feedback_category_dict[y_predict_dummy[0]]
print(f'Predicted category: {predicted_category}')

Predicted category: workload


In [46]:
joblib.dump(rf,'feedback_model.pkl')

['feedback_model.pkl']

In [27]:
# load_model=joblib.load('feedback_model.pkl')
# dummy_data_input=input()
# dummy_data=pd.DataFrame({'Feedback':[dummy_data_input]})
# dummy_data['Feedback']=dummy_data['Feedback'].apply(preprocess)

# dummy_tfidf=v.transform(dummy_data['Feedback'])
# dummy_data['Feedback']

# y_predict_dummy=load_model.predict(dummy_tfidf)
# print(y_predict_dummy)
# feedback_category_dict = {
#     0: 'team',
#     1: 'workload',
#     2: 'culture',
#     3: 'opportunities',
#     4: 'values',
#     5: 'management',
#     6: 'training',
#     7: 'compensation',
#     8: 'office'
# }

# predicted_category = feedback_category_dict[y_predict_dummy[0]]
# print(f'Predicted category: {predicted_category}')